In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df = pd.read_csv('/content/drive/MyDrive/Kaggle/Titanic-dataset/Titanic/train.csv')
print("Full train dataset shape is {}".format(df.shape))

Full train dataset shape is (891, 12)


In [8]:
def handle_non_numerical_data(df):

    columns = df.columns.values

    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        #print(column,df[column].dtype)
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:

            column_contents = df[column].values.tolist()
            #finding just the uniques
            unique_elements = set(column_contents)
            # great, found them.
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1
            df[column] = list(map(convert_to_int,df[column]))

    return df


In [10]:
y_target = df['Survived']
# Y_target.reshape(len(Y_target),1)
x_train = df[['Pclass', 'Age', 'Sex', 'SibSp', 'Parch', 'Fare','Embarked', 'Ticket']]

x_train = handle_non_numerical_data(x_train)
x_train.head()


<ipython-input-8-76cd39045a6a>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = list(map(convert_to_int,df[column]))


,Pclass,Age,Sex,SibSp,Parch,Fare,Embarked,Ticket
0,3,22.0,1,1,0,7.2500,0,396
1,1,38.0,0,1,0,71.2833,1,289
2,3,26.0,0,0,0,7.9250,0,150
3,1,35.0,0,1,0,53.1000,0,76
4,3,35.0,1,0,0,8.0500,0,52


In [11]:
fare = pd.DataFrame(x_train['Fare'])
# Normalizing
min_max_scaler = preprocessing.MinMaxScaler()
newfare = min_max_scaler.fit_transform(fare)
x_train['Fare'] = newfare
x_train

<ipython-input-11-7731635fa139>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['Fare'] = newfare


,Pclass,Age,Sex,SibSp,Parch,Fare,Embarked,Ticket
0,3,22.0,1,1,0,0.014151,0,396
1,1,38.0,0,1,0,0.139136,1,289
2,3,26.0,0,0,0,0.015469,0,150
3,1,35.0,0,1,0,0.103644,0,76
4,3,35.0,1,0,0,0.015713,0,52
...,...,...,...,...,...,...,...,...
886,2,27.0,1,0,0,0.025374,0,399
887,1,19.0,0,0,0,0.058556,0,386
888,3,NaN,0,1,2,0.045771,0,653
889,1,26.0,1,0,0,0.058556,1,314


In [12]:
x_train.isnull().sum()

Pclass        0
Age         177
Sex           0
SibSp         0
Parch         0
Fare          0
Embarked      0
Ticket        0
dtype: int64

In [13]:
# Fill the NAN values with the median values in the datasets
x_train['Age'] = x_train['Age'].fillna(x_train['Age'].median())
print("Number of NULL values" , x_train['Age'].isnull().sum())
print(x_train.head(3))

Number of NULL values 0
   Pclass   Age  Sex  SibSp  Parch      Fare  Embarked  Ticket
0       3  22.0    1      1      0  0.014151         0     396
1       1  38.0    0      1      0  0.139136         1     289
2       3  26.0    0      0      0  0.015469         0     150


<ipython-input-13-7491f431d8b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['Age'] = x_train['Age'].fillna(x_train['Age'].median())


In [14]:
x_train['Sex'] = x_train['Sex'].replace('male', 0)
x_train['Sex'] = x_train['Sex'].replace('female', 1)
# print(type(x_train))
corr = x_train.corr()
corr.style.background_gradient()

<ipython-input-14-9c798e5af0cf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['Sex'] = x_train['Sex'].replace('male', 0)
<ipython-input-14-9c798e5af0cf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train['Sex'] = x_train['Sex'].replace('female', 1)


,Pclass,Age,Sex,SibSp,Parch,Fare,Embarked,Ticket
Pclass,1.000000,-0.339898,0.131900,0.083081,0.018443,-0.549500,0.028566,0.034032
Age,-0.339898,1.000000,0.081163,-0.233296,-0.172482,0.096688,0.007505,-0.080710
Sex,0.131900,0.081163,1.000000,-0.114631,-0.245489,-0.182333,-0.128526,-0.041933
SibSp,0.083081,-0.233296,-0.114631,1.000000,0.414838,0.159651,-0.063794,-0.033758
Parch,0.018443,-0.172482,-0.245489,0.414838,1.000000,0.216225,-0.082144,0.057966
Fare,-0.549500,0.096688,-0.182333,0.159651,0.216225,1.000000,0.071010,-0.029318
Embarked,0.028566,0.007505,-0.128526,-0.063794,-0.082144,0.071010,1.000000,-0.014863
Ticket,0.034032,-0.080710,-0.041933,-0.033758,0.057966,-0.029318,-0.014863,1.000000


In [15]:
def plot_corr(df,size=10):


    corr = df.corr()
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)), corr.columns);
    plt.yticks(range(len(corr.columns)), corr.columns);
# plot_corr(x_train)
x_train.corr()
corr.style.background_gradient()

,Pclass,Age,Sex,SibSp,Parch,Fare,Embarked,Ticket
Pclass,1.000000,-0.339898,0.131900,0.083081,0.018443,-0.549500,0.028566,0.034032
Age,-0.339898,1.000000,0.081163,-0.233296,-0.172482,0.096688,0.007505,-0.080710
Sex,0.131900,0.081163,1.000000,-0.114631,-0.245489,-0.182333,-0.128526,-0.041933
SibSp,0.083081,-0.233296,-0.114631,1.000000,0.414838,0.159651,-0.063794,-0.033758
Parch,0.018443,-0.172482,-0.245489,0.414838,1.000000,0.216225,-0.082144,0.057966
Fare,-0.549500,0.096688,-0.182333,0.159651,0.216225,1.000000,0.071010,-0.029318
Embarked,0.028566,0.007505,-0.128526,-0.063794,-0.082144,0.071010,1.000000,-0.014863
Ticket,0.034032,-0.080710,-0.041933,-0.033758,0.057966,-0.029318,-0.014863,1.000000


In [16]:
# Dividing the data into train and test data set
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_target, test_size = 0.4, random_state = 40)

In [17]:
clf = RandomForestClassifier()
clf.fit(X_train, Y_train)

RandomForestClassifier()

In [18]:
print(clf.predict(X_test))
print("Accuracy: ",clf.score(X_test, Y_test))

[1 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 1 1
 0 1 1 0 0 0 1 1 0 0 1 0 0 1 0 1 1 1 1 0 1 1 0 0 0 0 1 0 1 1 1 0 0 1 1 0 0
 0 0 0 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0
 1 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1 0 1 1 1 1 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0
 1 0 0 0 1 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 1
 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 1 1 1 1
 0 1 0 1 1 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1
 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0
 1 0 0 1 0 0 1 1 0 1 0 1 1 1 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0 1 1 0 1 1 1
 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 1 0]
Accuracy:  0.7955182072829131


In [20]:
## Testing the model.
test_data = pd.read_csv('/content/drive/MyDrive/Kaggle/Titanic-dataset/Titanic/test.csv')
test_data.head(3)
# test_data.isnull().sum()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [21]:
### Preprocessing on the test data
test_data = test_data[['Pclass', 'Age', 'Sex', 'SibSp', 'Parch', 'Fare', 'Ticket', 'Embarked']]
test_data = handle_non_numerical_data(test_data)

fare = pd.DataFrame(test_data['Fare'])
min_max_scaler = preprocessing.MinMaxScaler()
newfare = min_max_scaler.fit_transform(fare)
test_data['Fare'] = newfare
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())
test_data['Age'] = test_data['Age'].fillna(test_data['Age'].median())
test_data['Sex'] = test_data['Sex'].replace('male', 0)
test_data['Sex'] = test_data['Sex'].replace('female', 1)
print(test_data.head())


   Pclass   Age  Sex  SibSp  Parch      Fare  Ticket  Embarked
0       3  34.5    1      0      0  0.015282     185         1
1       3  47.0    0      1      0  0.013663     171         0
2       2  62.0    1      0      0  0.018909     257         1
3       3  27.0    1      0      0  0.016908     125         0
4       3  22.0    0      1      1  0.023984     118         0


<ipython-input-8-76cd39045a6a>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = list(map(convert_to_int,df[column]))
<ipython-input-21-3973be4c0aae>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Fare'] = newfare
<ipython-input-21-3973be4c0aae>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [25]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"samikshyamizar","key":"ca3879ecb470dcff787acb02fb9a02f4"}'}

In [26]:
!mkdir -p /root/.kaggle
!mv kaggle.json /root/.kaggle/


In [28]:
! kaggle competitions submit -c Titanic -f /content/drive/MyDrive/Kaggle/Titanic-dataset/Titanic/gender_submission.csv -m gender_submission.csv

100% 3.18k/3.18k [00:01<00:00, 1.74kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster